# Создание AI-агентов с постоянной памятью с использованием Cognee

Этот ноутбук демонстрирует, как создавать интеллектуальных AI-агентов с продвинутыми возможностями памяти, используя [**cognee**](https://www.cognee.ai/) — открытый инструмент для работы с памятью AI, который объединяет графы знаний, семантический поиск и управление сессиями для создания контекстно-осведомленных AI-систем.

## 🎯 Цели обучения

К концу этого урока вы научитесь:
- **Создавать графы знаний на основе эмбеддингов**: Преобразовывать неструктурированный текст в структурированные, доступные для запросов знания
- **Реализовывать память сессий**: Создавать многотуровые диалоги с автоматическим сохранением контекста
- **Сохранять диалоги**: При необходимости сохранять важные взаимодействия в долгосрочной памяти для последующего использования
- **Запрашивать с использованием естественного языка**: Получать доступ к историческому контексту и использовать его в новых диалогах
- **Визуализировать память**: Исследовать связи в графе знаний вашего агента


## 🏗️ Что вы создадите

В этом руководстве мы создадим **Ассистента для кодирования** с постоянной памятью, который:

### 1. **Создание базы знаний**
   - Загружает информацию о профиле разработчика и его навыках
   - Обрабатывает принципы программирования на Python и лучшие практики
   - Сохраняет историю разговоров между разработчиками и AI-ассистентами

### 2. **Учет контекста в разговорах**
   - Сохраняет контекст при ответах на несколько вопросов в одной сессии
   - Автоматически кэширует каждую пару вопрос/ответ для эффективного извлечения
   - Предоставляет связные и контекстуальные ответы, основываясь на истории разговора

### 3. **Долгосрочная память**
   - Сохраняет важные разговоры в долгосрочную память
   - Извлекает релевантные воспоминания из базы знаний и прошлых сессий для новых взаимодействий
   - Создает растущую базу знаний, которая со временем становится лучше

### 4. **Интеллектуальное извлечение памяти**
   - Использует семантический поиск с учетом графов для нахождения релевантной информации во всей сохраненной базе знаний
   - Фильтрует поиск по подгруппам данных (информация о разработчике vs. принципы)
   - Объединяет несколько источников данных для предоставления комплексных ответов


## 📋 Предварительные требования и настройка

### Системные требования

Перед началом убедитесь, что у вас есть:

1. **Среда Python**
   - Python 3.9 или выше
   - Виртуальная среда (рекомендуется)
   
2. **Кэш Redis** (требуется для управления сессиями)
   - Локальный Redis: `docker run -d -p 6379:6379 redis`
   - Или используйте управляемый сервис Redis
   
3. **Доступ к API LLM**
   - Ключ API OpenAI или других провайдеров (см. [документацию](https://docs.cognee.ai/setup-configuration/llm-providers))

4. **Конфигурация базы данных**
   - По умолчанию настройка не требуется. Cognee использует файловые базы данных (LanceDB и Kuzu)
   - При желании вы можете настроить Azure AI Search как хранилище векторов (см. [документацию](https://github.com/topoteretes/cognee-community/tree/main/packages/vector/azureaisearch))

### Конфигурация окружения

Создайте файл `.env` в каталоге вашего проекта с следующими переменными:

```ini
# LLM Configuration (Required)
LLM_API_KEY=your-openai-api-key-here

# Cache Configuration (Required for Sessions)
CACHING=true  # Must be enabled for session history

```


## 🏛️ Понимание архитектуры памяти Cognee

### Как работает Cognee

Cognee предоставляет сложную систему памяти, которая выходит за рамки простого хранения ключ-значение:

```
┌──────────────────────────┐
│      30+ data sources    │
└───────────┬──────────────┘
            │
            ▼
┌──────────────────────────────────────────┐
│  Dynamically evolving memory layers      │
│                                          │
│  ┌────────────────────────────────────┐  │
│  │ Knowledge Graph in Graph Database  │  │
│  └────────────────────────────────────┘  │
│  ┌────────────────────────────────────┐  │
│  │ Embeddings in Vector Store         │  │
│  │   (e.g., Azure AI Search)          │  │
│  └────────────────────────────────────┘  │
└───────────┬──────────────────────────────┘
            │                      ▲   
            ▼                      │(optional)
┌────────────────┐           ┌────────────────┐
│     cognee     │(optional) │ Cognee Session │
│    retrievers  │──────────▶│     Cache      │
│                │           │    (Redis)     │
└───────┬────────┘           └────────────────┘
        ▲
        │
┌──────────────────────────┐
│          Agents          │
└──────────────────────────┘

```

### Основные компоненты:

1. **Граф знаний**: Хранит сущности, отношения и семантические связи
2. **Векторные встраивания**: Обеспечивают семантический поиск по всей сохраненной информации
3. **Кэш сессии**: Сохраняет контекст разговора внутри и между сессиями
4. **NodeSets**: Организуют данные в логические категории для целенаправленного извлечения

### Типы памяти в этом руководстве:

- **Постоянная память**: Долгосрочное хранение в графе знаний
- **Память сессии**: Временный контекст разговора в кэше Redis
- **Семантическая память**: Поиск по сходству на основе векторов по всем данным


## 📦 Установка необходимых пакетов

Установите Cognee с поддержкой Redis для управления сессиями:


In [ ]:
!pip install --quiet "cognee[redis]==0.4.0"

## 🔧 Инициализация окружения и загрузка библиотек

Убедитесь, что:
1. Redis запущен (например, через Docker: `docker run -d -p 6379:6379 redis`)
2. Переменные окружения установлены перед импортом модулей кэша
3. При необходимости перезапустите ядро и выполните ячейки по порядку

Следующая ячейка выполнит:
1. Загрузку переменных окружения из `.env`
2. Настройку Cognee с вашими параметрами LLM
3. Включение кэширования для управления сессиями
4. Проверку правильного подключения всех компонентов


In [ ]:
import os
from pathlib import Path

from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# cognee Configuration
os.environ["LLM_API_KEY"] = os.getenv("LLM_API_KEY", None)
os.environ["CACHING"] = os.getenv("CACHING", "true")


import cognee

print(f"Cognee version: {cognee.__version__}")
print(f"CACHING: {os.environ.get('CACHING')}")
print(f"LLM_API_KEY: {os.environ.get('LLM_API_KEY')}")

## 📁 Настройка каталогов хранения

Cognee использует два отдельных каталога для своей работы:
- **Корень данных**: Хранит загруженные документы и обработанные данные
- **Системный корень**: Содержит базу данных графа знаний и системные метаданные

Для этого урока мы создадим изолированные каталоги следующим образом:


In [ ]:
DATA_ROOT = Path('.data_storage').resolve()
SYSTEM_ROOT = Path('.cognee_system').resolve()

DATA_ROOT.mkdir(parents=True, exist_ok=True)
SYSTEM_ROOT.mkdir(parents=True, exist_ok=True)

cognee.config.data_root_directory(str(DATA_ROOT))
cognee.config.system_root_directory(str(SYSTEM_ROOT))

print(f"Data root: {DATA_ROOT}")
print(f"System root: {SYSTEM_ROOT}")

## 🧹 Сброс состояния памяти

Прежде чем мы начнем создавать нашу систему памяти, давайте убедимся, что мы начинаем с чистого листа.

> 💡 **Совет**: Вы можете пропустить этот шаг, если хотите сохранить существующие данные памяти из предыдущих запусков, когда будете использовать этот ноутбук позже.


In [ ]:
await cognee.prune.prune_data()
await cognee.prune.prune_system(metadata=True)
print('Cleared previous Cognee state.')

## 📚 Часть 1: Создание базы знаний

### Источники данных для нашего помощника разработчика

Мы будем использовать три типа данных для создания всеобъемлющей базы знаний:

1. **Профиль разработчика**: Личный опыт и технический бэкграунд
2. **Лучшие практики Python**: Дзен Python с практическими рекомендациями
3. **Исторические беседы**: Прошлые сессии вопросов и ответов между разработчиками и AI-ассистентами

Эти разнообразные данные позволяют нашему агенту:
- Понимать технический контекст пользователя
- Применять лучшие практики в рекомендациях
- Учиться на предыдущих успешных взаимодействиях


In [ ]:
developer_intro = (
  "Hi, I'm an AI/Backend engineer. "
  "I build FastAPI services with Pydantic, heavy asyncio/aiohttp pipelines, "
  "and production testing via pytest-asyncio. "
  "I've shipped low-latency APIs on AWS, Azure, and GoogleCloud."
)

python_zen_principles = (
  """
    # The Zen of Python: Practical Guide

    ## Overview
    Use these principles as a checklist during design, coding, and reviews.

    ## Key Principles With Guidance

    ### 1. Beautiful is better than ugly
    Prefer descriptive names, clear structure, and consistent formatting.

    ### 2. Explicit is better than implicit
    Be clear about behavior, imports, and types.
    ```python
    from datetime import datetime, timedelta

    def get_future_date(days_ahead: int) -> datetime:
        return datetime.now() + timedelta(days=days_ahead)
    ```

    ### 3. Simple is better than complex
    Choose straightforward solutions first.

    ### 4. Complex is better than complicated
    When complexity is needed, organize it with clear abstractions.

    ### 5. Flat is better than nested
    Use early returns to reduce indentation.

    ## Modern Python Tie-ins
    - Type hints reinforce explicitness
    - Context managers enforce safe resource handling
    - Dataclasses improve readability for data containers

    ## Quick Review Checklist
    - Is it readable and explicit?
    - Is this the simplest working solution?
    - Are errors explicit and logged?
    - Are modules/namespaces used appropriately?
  """
)

human_agent_conversations = (
  """
  "conversations": [
      {
        "id": "conv_001",
        "timestamp": "2024-01-15T10:30:00Z",
        "topic": "async/await patterns",
        "user_query": "I'm building a web scraper that needs to handle thousands of URLs concurrently. What's the best way to structure this with asyncio?",
        "assistant_response": "Use asyncio with aiohttp, a semaphore to cap concurrency, TCPConnector for connection pooling, context managers for session lifecycle, and robust exception handling for failed requests.",
        "code_context": {
          "file": "scraper.py",
          "language": "python",
          "patterns_discussed": ["async/await", "context_managers", "semaphores", "aiohttp", "error_handling"]
        },
        "follow_up_questions": [
          "How do I add retry logic for failed requests?",
          "What's the best way to parse the scraped HTML content?"
        ]
      },
      {
        "id": "conv_002",
        "timestamp": "2024-01-16T14:20:00Z",
        "topic": "dataclass vs pydantic",
        "user_query": "When should I use dataclasses vs Pydantic models? I'm building an API and need to handle user input validation.",
        "assistant_response": "For API input/output, prefer Pydantic: it provides runtime validation, type coercion, JSON serialization, enums for roles, field constraints, and custom validators; integrates cleanly with FastAPI for automatic request validation and error reporting.",
        "code_context": {
          "file": "models.py",
          "language": "python",
          "patterns_discussed": ["pydantic", "dataclasses", "validation", "fastapi", "type_hints", "enums"]
        },
        "follow_up_questions": [
          "How do I handle nested validation with Pydantic?",
          "Can I use Pydantic with SQLAlchemy models?"
        ]
      },
      {
        "id": "conv_003",
        "timestamp": "2024-01-17T09:15:00Z",
        "topic": "testing patterns",
        "user_query": "I'm struggling with testing async code and database interactions. What's the best approach for pytest with async functions?",
        "assistant_response": "Recommended using pytest-asyncio, async fixtures, and an isolated test database or mocks to reliably test async functions and database interactions in FastAPI.",
        "code_context": {
          "file": "test_users.py",
          "language": "python",
          "patterns_discussed": ["pytest", "async_testing", "fixtures", "mocking", "database_testing", "fastapi_testing"]
        },
        "follow_up_questions": [
          "How do I test WebSocket connections?",
          "What's the best way to test database migrations?"
        ]
      },
      {
        "id": "conv_004",
        "timestamp": "2024-01-18T16:45:00Z",
        "topic": "performance optimization",
        "user_query": "My FastAPI app is getting slow with large datasets. How can I optimize database queries and response times?",
        "assistant_response": "Suggested optimizing database queries (indexes, pagination, selecting only needed columns), adding caching, streaming or chunked responses for large datasets, background tasks for heavy work, and monitoring to find bottlenecks.",
        "code_context": {
          "file": "optimizations.py",
          "language": "python",
          "patterns_discussed": ["performance_optimization", "caching", "database_optimization", "async_patterns", "monitoring"]
        },
        "follow_up_questions": [
          "How do I implement database connection pooling properly?",
          "What's the best way to handle memory usage with large datasets?"
        ]
      },
      {
        "id": "conv_005",
        "timestamp": "2024-01-19T11:30:00Z",
        "topic": "error handling and logging",
        "user_query": "I need to implement proper error handling and logging across my Python application. What's the best approach for production-ready error management?",
        "assistant_response": "Proposed centralized error handling with custom exceptions, structured logging, FastAPI middleware or decorators, and integration points for external monitoring/alerting tools.",
        "code_context": {
          "file": "error_handling.py",
          "language": "python",
          "patterns_discussed": ["error_handling", "logging", "exceptions", "middleware", "decorators", "fastapi"]
        },
        "follow_up_questions": [
          "How do I integrate this with external monitoring tools like Sentry?",
          "What's the best way to handle errors in background tasks?"
        ]
      }
    ],
    "metadata": {
      "total_conversations": 5,
      "date_range": "2024-01-15 to 2024-01-19",
      "topics_covered": [
        "async/await patterns",
        "dataclass vs pydantic",
        "testing patterns",
        "performance optimization",
        "error handling and logging"
      ],
      "code_patterns_discussed": [
        "asyncio", "aiohttp", "semaphores", "context_managers",
        "pydantic", "fastapi", "type_hints", "validation",
        "pytest", "async_testing", "fixtures", "mocking",
        "performance_optimization", "caching", "database_optimization",
        "error_handling", "logging", "exceptions", "middleware"
      ],
      "difficulty_levels": {
        "beginner": 1,
        "intermediate": 2,
        "advanced": 2
      }
    }
  """
)

## 🔄 Преобразование данных в граф знаний

Теперь мы преобразуем наш необработанный текст в структурированную память. Этот процесс:

1. **Добавляет данные в NodeSets**: Организует информацию в логические категории
   - `developer_data`: Профиль разработчика и разговоры
   - `principles_data`: Лучшие практики и рекомендации по Python

2. **Запускает Cognify Pipeline**: Извлекает сущности, отношения и создает эмбеддинги
   - Определяет ключевые концепции
   - Создает семантические связи между связанными данными
   - Генерирует векторные эмбеддинги

Это может занять некоторое время, пока LLM обрабатывает текст и строит структуру графа:


In [ ]:
await cognee.add(developer_intro, node_set=["developer_data"])
await cognee.add(human_agent_conversations, node_set=["developer_data"])
await cognee.add(python_zen_principles, node_set=["principles_data"])

await cognee.cognify()

## 📊 Визуализация графа знаний

Давайте изучим структуру нашего графа знаний. Визуализация показывает:
- **Узлы**: Сущности, извлеченные из текста (концепции, технологии, люди)
- **Ребра**: Отношения и связи между сущностями
- **Кластеры**: Связанные концепции, сгруппированные по семантическому сходству

Откройте сгенерированный HTML-файл в вашем браузере, чтобы интерактивно исследовать граф:


In [ ]:
from cognee import visualize_graph
await visualize_graph('./visualization_1.html')

## 🧠 Обогащение памяти с помощью Memify

Функция `memify()` анализирует граф знаний и генерирует интеллектуальные правила о данных. Этот процесс:
- Определяет шаблоны и лучшие практики
- Создает практические рекомендации на основе контента
- Устанавливает связи между различными областями знаний

Эти правила помогают агенту принимать более обоснованные решения при ответах на вопросы. Захват второй визуализации позволяет сравнить, как граф становится более плотным после обогащения.


In [ ]:
await cognee.memify()

await visualize_graph('./visualization_2.html')

## 🔍 Часть 2: Интеллектуальное извлечение информации

### Демонстрация 1: Интеграция знаний из разных документов

Теперь, когда наш граф знаний создан, давайте проверим, как Cognee объединяет информацию из нескольких источников для ответа на сложные вопросы.

Первый запрос демонстрирует:
- **Семантическое понимание**: Поиск релевантных концепций, даже если они явно не упомянуты
- **Перекрестные ссылки**: Сочетание профиля разработчика с принципами Python
- **Контекстуальное рассуждение**: Применение лучших практик к конкретным реализациям

### Демонстрация 2: Фильтрованный поиск с NodeSets

Второй запрос показывает, как нацелиться на определенные подмножества графа знаний:
- Использует параметр `node_name` для поиска только в пределах `principles_data`
- Предоставляет сфокусированные ответы из конкретной области знаний
- Полезно, когда требуется информация из определенной области


In [ ]:
# demonstrate cross-document knowledge retrieval from multiple data sources
from cognee.modules.search.types import SearchType

results = await cognee.search(
    query_text="How does my AsyncWebScraper implementation align with Python's design principles?",
    query_type=SearchType.GRAPH_COMPLETION,
)
print("Python Pattern Analysis:", results)

# demonstrate filtered search using NodeSet to query only specific subsets of memory
from cognee.modules.engine.models.node_set import NodeSet
results = await cognee.search(
    query_text="How should variables be named?",
    query_type=SearchType.GRAPH_COMPLETION,
    node_type=NodeSet,
    node_name=["principles_data"],
)
print("Filtered search result:", results)

## 🔐 Часть 3: Настройка управления сессиями

### Включение памяти диалога

Управление сессиями важно для сохранения контекста в ходе нескольких взаимодействий. Здесь мы:

1. **Инициализируем пользовательский контекст**: Создаем или извлекаем профиль пользователя для отслеживания сессии
2. **Настраиваем механизм кэширования**: Подключаемся к Redis для хранения истории диалога
3. **Включаем переменные сессии**: Настраиваем переменные контекста, которые сохраняются между запросами

> ⚠️ **Важно**: Для этого требуется запущенный Redis и `CACHING=true` в вашем окружении


In [ ]:
from cognee.modules.users.methods import get_default_user
from cognee.context_global_variables import set_session_user_context_variable 
from cognee.infrastructure.databases.cache import get_cache_engine

user = await get_default_user()
await set_session_user_context_variable(user)
print(f"Using user id: {getattr(user, 'id', 'unknown')}")

cache_engine = get_cache_engine()
if cache_engine is None:
    raise RuntimeError('Cache engine is not available. Double-check your cache configuration.')
print('Session cache is ready.')


## 🛠️ Вспомогательная функция: Просмотр истории сессии

Эта утилита позволяет нам изучить историю разговоров, сохраненную в Redis. Она полезна для:
- Отладки управления сессиями
- Проверки, что разговоры сохраняются в кэше
- Понимания, какой контекст доступен агенту


In [ ]:
async def show_history(session_id: str) -> None:
    # Let's check the cache directly
    cache_engine = get_cache_engine()
    if cache_engine:
        # Try to get history directly from cache
        user_id = str(user.id) if hasattr(user, 'id') else None
        if user_id:
            history_entries = await cache_engine.get_latest_qa(user_id, session_id, last_n=10)
            print(f"\nDirect cache query for user_id={user_id}, session_id={session_id}:")
            print(f"Found {len(history_entries)} entries")
            if history_entries:
                for i, entry in enumerate(history_entries, 1):
                    print(f"\nEntry {i}:")
                    print(f"  Question: {entry.get('question', 'N/A')[:100]}...")
                    print(f"  Answer: {entry.get('answer', 'N/A')[:100]}...")
        else:
            print("No user_id available")


## Сессия 1: Лаборатория поддержки Async — Первый вопрос

Начните сессию `async-support-lab`, задав вопрос о подходящих для телеметрии паттернах asyncio для масштабного веб-скрейпера. Граф уже знаком с asyncio, aiohttp и практиками мониторинга, поэтому ответ должен отражать предыдущие обсуждения, адаптируясь к новому запросу.


In [ ]:
session_1 = "async-support-lab"

result = await cognee.search(
    query_type=SearchType.GRAPH_COMPLETION,
    query_text="I'm building a web scraper that hits thousands of URLs concurrently. What's a reliable asyncio pattern with telemetry?",
    session_id=session_1
)

## Проверка памяти сессии 1 после первого обмена

Выполнение команды `show_history(session_1)` сразу после первоначального вопроса подтверждает, что Cognee записал как запрос, так и ответ в Redis. Вы должны увидеть одну запись с рекомендацией по параллельности.


In [ ]:
await show_history(session_1)

## Сессия 1: Продолжение обсуждения моделей данных

Далее мы задаем вопрос: «Когда следует выбирать dataclasses, а когда Pydantic?» с использованием того же идентификатора сессии. Cognee должен объединить принципы Python с предыдущими обсуждениями FastAPI, чтобы дать детализированный совет, демонстрируя, что контекст сохраняется в рамках именованной сессии.


In [ ]:
result = await cognee.search(
    query_type=SearchType.GRAPH_COMPLETION,
    query_text="When should I pick dataclasses versus Pydantic for this work?",
    session_id=session_1
)

## Подтвердите, что история сессии 1 содержит оба хода

Еще один вызов `show_history(session_1)` должен показать две записи вопросов и ответов. Это соответствует шагу "воспроизведение памяти" в лаборатории Mem0 и доказывает, что дополнительные ходы продолжают тот же самый транскрипт.


In [ ]:
await show_history(session_1)

## Сессия 2: Обсуждение дизайна — Новая сессия

Чтобы показать изоляцию между потоками, мы запускаем `design-review-session` и запрашиваем рекомендации по ведению логов для анализа инцидентов. Хотя база знаний остается той же, новый идентификатор сессии позволяет хранить стенограммы отдельно.


In [ ]:
session_2 = "design-review-session"

result = await cognee.search(
    query_type=SearchType.GRAPH_COMPLETION,
    query_text="We're drafting logging guidance for incident reviews. Capture the key principles please.",
    session_id=session_2
)

## Обзор сессии 2 История

`show_history(session_2)` должен отображать только пару запрос/ответ для обзора дизайна. Сравните это с Сессией 1, чтобы подчеркнуть, как Cognee сохраняет независимые стенограммы, одновременно используя общий граф знаний.


In [ ]:
await show_history(session_2)

## Резюме

Поздравляем! Вы только что добавили вашему помощнику по программированию настоящий слой долгосрочной памяти, работающий на основе Cognee.

В этом руководстве вы взяли исходный контент разработчика (код, документацию, чаты) и преобразовали его в граф + векторную память, которую ваш агент может искать, анализировать и постоянно улучшать.

Чему вы научились:

1. **От текста к памяти ИИ**: Как Cognee обрабатывает неструктурированные данные и превращает их в интеллектуальную, доступную для поиска память, используя комбинированную архитектуру векторов и графа знаний.

2. **Обогащение графа с помощью memify**: Как выйти за рамки базового создания графа и использовать memify для добавления производных фактов и более богатых связей поверх существующего графа.

3. **Различные стратегии поиска**: Как выполнять запросы к памяти с использованием различных типов поиска (вопросы с учетом графа, завершение в стиле RAG, инсайты, исходные фрагменты, поиск кода и т.д.) в зависимости от потребностей вашего агента.

4. **Визуальное исследование**: Как проверять и отлаживать то, что создал Cognee, с помощью визуализации графов и интерфейса Cognee, чтобы вы могли увидеть, как структурированы знания.

5. **Память с учетом сеанса**: Как сочетать контекст каждого сеанса с постоянной семантической памятью, чтобы агенты могли запоминать между запусками, не допуская утечки информации между пользователями.


## Основные моменты
1. Память как граф знаний, поддерживаемый эмбеддингами

    - **Структурированное понимание**: Cognee объединяет хранилище векторов и графовое хранилище, чтобы ваши данные были одновременно доступны для поиска по смыслу и связаны отношениями. По умолчанию Cognee использует файловые базы данных (LanceDB для векторов, Kuzu для графовой базы данных).

    - **Учёт отношений при поиске**: Ответы могут основываться не только на «похожем тексте», но и на том, как связаны сущности.

    - **Живая память**: Слой памяти развивается, растёт и остаётся доступным для запросов как единый связанный граф.

2. Режимы поиска и рассуждения
    - **Гибридный поиск**: поиск объединяет сходство векторов, структуру графа и рассуждения LLM, начиная от поиска по сырым фрагментам до ответов с учётом графа.

    - **Подбор режима под задачу**: Используйте режимы в стиле завершения, когда нужны ответы на естественном языке, и режимы фрагментов/резюме/графа, когда вашему агенту требуется сырой контекст или возможность самостоятельно рассуждать.

3. Персонализированные агенты, учитывающие контекст сессии
    - **Контекст сессии + долгосрочная память**: Cognee разделяет краткосрочный контекст «ветки» и долгосрочную память на уровне пользователя или организации.

## Применение в реальном мире

1. **Вертикальные AI-агенты**

    Используйте шаблон из этого блокнота для создания умных помощников, которые работают на основе Cognee как ядра для поиска и рассуждений:

- **Помощники для разработчиков**: Помощь в код-ревью, анализе инцидентов и архитектурных решениях, которые охватывают код, API, проектную документацию и тикеты как единый граф памяти.

- **Помощники для клиентов**: Агенты поддержки или успеха, которые используют продуктовую документацию, FAQ, заметки CRM и прошлые тикеты с учётом графа и цитируемыми ответами.

- **Внутренние экспертные помощники**: Помощники по политике, юридическим вопросам или безопасности, которые рассуждают на основе взаимосвязанных правил, руководств и исторических решений, а не отдельных PDF.

    Cognee позиционируется как постоянная, точная память для AI-агентов, предоставляя живой граф знаний, который интегрируется с вашим агентом и заменяет случайные комбинации хранилищ векторов и пользовательского графового кода.

2. **Объединение разрозненных данных в одну память**

    Тот же подход помогает создать единый слой памяти из разбросанных источников:

- **От изолированных данных к единому графу**: Загружайте структурированные (например, базы данных) и неструктурированные данные (например, документы, чаты) в единый граф, поддерживаемый эмбеддингами, вместо отдельных индексов для каждой системы.

- **Рассуждения с учётом источников и цитат**: Выполняйте многошаговые рассуждения по всем данным — «объединяйте» логи, метрики и документы через граф — и всё равно возвращайте обоснованные ответы с указанием источников.

- **Центры знаний**: В таких областях, как банковское дело или образование, Cognee уже используется для объединения PDF, внутренних систем и данных приложений в единый граф знаний с векторами, чтобы агенты могли отвечать на вопросы с точным, цитируемым контекстом.

## Следующие шаги

Вы реализовали основной цикл памяти. Вот естественные расширения, которые вы можете попробовать самостоятельно (см. [документацию Cognee](https://docs.cognee.ai/) для подробностей):

1. **Экспериментируйте с временной осведомлённостью**: Включите временную функцию Cognee, чтобы извлекать события и временные метки из текста.

2. **Внедрите рассуждения, основанные на онтологии**: Определите онтологию OWL для вашей области. Используйте поддержку онтологии Cognee, чтобы извлечённые сущности и отношения были привязаны к этой схеме, улучшая качество графа и ответы, специфичные для вашей области.

3. **Добавьте обратную связь**: Позвольте Cognee корректировать веса графовых связей на основе реальной обратной связи от пользователей, чтобы поиск улучшался со временем, а не оставался статичным.

4. **Настройте персонализацию и поведение сессий**: Используйте идентификаторы пользователей, арендаторов и наборы данных, чтобы предоставить каждому человеку или команде их собственный вид на общий механизм памяти.

5. **Масштабируйте для более сложных агентов**: Подключите Cognee к фреймворкам агентов, чтобы создавать системы с несколькими агентами, которые все используют общий слой памяти. *Microsoft Agent Framework x Cognee plugin скоро будет доступен.*


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Отказ от ответственности**:  
Этот документ был переведен с использованием сервиса автоматического перевода [Co-op Translator](https://github.com/Azure/co-op-translator). Несмотря на наши усилия обеспечить точность, автоматические переводы могут содержать ошибки или неточности. Оригинальный документ на его родном языке следует считать авторитетным источником. Для получения критически важной информации рекомендуется профессиональный перевод человеком. Мы не несем ответственности за любые недоразумения или неправильные интерпретации, возникающие в результате использования данного перевода.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
